In [1]:
import polars as pl

In [83]:
df = pl.read_csv('CardiacArrestPrognos_DATA_LABELS_2025-01-07_2344.csv', schema_overrides={
    'Date of arrest': pl.Datetime,
    'EEG start': pl.Datetime,
    'EEG end': pl.Datetime,
    'Discharge Date': pl.Datetime
})

In [105]:
final = pl.concat([
    df.group_by(['Site', 'CPC at Discharge']).agg([
        pl.col('Age').mean().round(2).cast(pl.String) + ' ± ' + pl.col('Age').std().round(2).cast(pl.String).alias('Age'),
        ((pl.col('Gender') == 'Male').sum().cast(pl.String) + ' (' + ((pl.col('Gender') == 'Male').sum() / pl.len() * 100).round(2).cast(pl.String) + ')').alias('Male')] +
        [
            ((pl.col(x) == 'Checked').sum().cast(pl.String) + ' (' + ((pl.col(x) == 'Checked').sum() / pl.len() * 100).round(2).cast(pl.String) + ')').alias(x.replace('(choice=', '').replace(')', '')) for x in df.columns if x.startswith('Race')
        ] +
        [
            ((pl.col('Ethnicity') == val).sum().cast(pl.String) + ' (' + ((pl.col('Ethnicity') == val).sum() / pl.len() * 100).round(2).cast(pl.String) + ')').alias('Ethnicity ' + val) for val in df['Ethnicity'].unique()
        ] +
        [
            (pl.col('out of hospital arrest includes arrest in a rehab facility or SNF') == 'Yes').sum().alias('Out of hospital arrest'),
            (pl.col('Temperature') == 'No Temperature Management').sum().alias('No temperature management'),
            (((pl.col('EEG start') - pl.col('Date of arrest')).filter(pl.col('out of hospital arrest includes arrest in a rehab facility or SNF') == 'Yes').mean() / pl.duration(days=1)).round(2).cast(pl.String) + ' ± ' + 
                ((pl.col('EEG start') - pl.col('Date of arrest')).filter(pl.col('out of hospital arrest includes arrest in a rehab facility or SNF') == 'Yes').std() / pl.duration(days=1)).round(2).cast(pl.String)).alias('Time to EEG (days)'),
            (((pl.col('Discharge Date') - pl.col('Date of arrest')).filter(pl.col('out of hospital arrest includes arrest in a rehab facility or SNF') == 'Yes').mean() / pl.duration(days=1)).round(2).cast(pl.String) + ' ± ' + 
                ((pl.col('Discharge Date') - pl.col('Date of arrest')).filter(pl.col('out of hospital arrest includes arrest in a rehab facility or SNF') == 'Yes').std() / pl.duration(days=1)).round(2).cast(pl.String)).alias('Hospital stay (days)'),
            (((pl.col('EEG end') - pl.col('EEG start')).filter(pl.col('out of hospital arrest includes arrest in a rehab facility or SNF') == 'Yes').mean() / pl.duration(days=1)).round(2).cast(pl.String) + ' ± ' + 
                ((pl.col('EEG end') - pl.col('EEG start')).filter(pl.col('out of hospital arrest includes arrest in a rehab facility or SNF') == 'Yes').std() / pl.duration(days=1)).round(2).cast(pl.String)).alias('Duration of EEG (days)'),
            pl.len().alias('Total patients')
        ]
    ),

    df.group_by('CPC at Discharge').agg([
        pl.lit('TOTAL').alias('Site'),
        pl.col('Age').mean().round(2).cast(pl.String) + ' ± ' + pl.col('Age').std().round(2).cast(pl.String).alias('Age'),
        ((pl.col('Gender') == 'Male').sum().cast(pl.String) + ' (' + ((pl.col('Gender') == 'Male').sum() / pl.len() * 100).round(2).cast(pl.String) + ')').alias('Male')] +
        [
            ((pl.col(x) == 'Checked').sum().cast(pl.String) + ' (' + ((pl.col(x) == 'Checked').sum() / pl.len() * 100).round(2).cast(pl.String) + ')').alias(x.replace('(choice=', '').replace(')', '')) for x in df.columns if x.startswith('Race')
        ] +
        [
            ((pl.col('Ethnicity') == val).sum().cast(pl.String) + ' (' + ((pl.col('Ethnicity') == val).sum() / pl.len() * 100).round(2).cast(pl.String) + ')').alias('Ethnicity ' + val) for val in df['Ethnicity'].unique()
        ] +
        [
            (pl.col('out of hospital arrest includes arrest in a rehab facility or SNF') == 'Yes').sum().alias('Out of hospital arrest'),
            (pl.col('Temperature') == 'No Temperature Management').sum().alias('No temperature management'),
            (((pl.col('EEG start') - pl.col('Date of arrest')).filter(pl.col('out of hospital arrest includes arrest in a rehab facility or SNF') == 'Yes').mean() / pl.duration(days=1)).round(2).cast(pl.String) + ' ± ' + 
                ((pl.col('EEG start') - pl.col('Date of arrest')).filter(pl.col('out of hospital arrest includes arrest in a rehab facility or SNF') == 'Yes').std() / pl.duration(days=1)).round(2).cast(pl.String)).alias('Time to EEG (days)'),
            (((pl.col('Discharge Date') - pl.col('Date of arrest')).filter(pl.col('out of hospital arrest includes arrest in a rehab facility or SNF') == 'Yes').mean() / pl.duration(days=1)).round(2).cast(pl.String) + ' ± ' + 
                ((pl.col('Discharge Date') - pl.col('Date of arrest')).filter(pl.col('out of hospital arrest includes arrest in a rehab facility or SNF') == 'Yes').std() / pl.duration(days=1)).round(2).cast(pl.String)).alias('Hospital stay (days)'),
            (((pl.col('EEG end') - pl.col('EEG start')).filter(pl.col('out of hospital arrest includes arrest in a rehab facility or SNF') == 'Yes').mean() / pl.duration(days=1)).round(2).cast(pl.String) + ' ± ' + 
                ((pl.col('EEG end') - pl.col('EEG start')).filter(pl.col('out of hospital arrest includes arrest in a rehab facility or SNF') == 'Yes').std() / pl.duration(days=1)).round(2).cast(pl.String)).alias('Duration of EEG (days)'),
            pl.len().alias('Total patients')
    ])
], how='diagonal')
display(final)
final.write_csv('demographics_table.csv')

Site,CPC at Discharge,Age,Male,Race American Indian/Alaska,Race Asian,Race Black/African American,Race Hispanic,Race Hawaiian,Race White,Race Mixed,Race Unknown,Ethnicity Hispanic,Ethnicity Unknown,Ethnicity Not Hispanic,Out of hospital arrest,No temperature management,Time to EEG (days),Hospital stay (days),Duration of EEG (days),Total patients
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,u32,u32,str,str,str,u32
"""BWH""","""dead""","""61.41 ± 16.75""","""140 (59.07)""","""0 (0.0)""","""7 (2.95)""","""44 (18.57)""","""0 (0.0)""","""0 (0.0)""","""148 (62.45)""","""0 (0.0)""","""38 (16.03)""","""26 (10.97)""","""36 (15.19)""","""175 (73.84)""",191,0,"""0.75 ± 0.46""","""7.2 ± 8.82""","""2.26 ± 1.51""",237
"""MGH""","""not dead""","""57.82 ± 17.17""","""111 (67.68)""","""0 (0.0)""","""8 (4.88)""","""14 (8.54)""","""0 (0.0)""","""0 (0.0)""","""112 (68.29)""","""0 (0.0)""","""30 (18.29)""","""17 (10.37)""","""27 (16.46)""","""120 (73.17)""",142,6,"""0.75 ± 0.53""","""29.05 ± 97.42""","""2.48 ± 2.38""",164
"""BIDMC""","""not dead""","""60.95 ± 15.53""","""26 (46.43)""","""0 (0.0)""","""0 (0.0)""","""3 (5.36)""","""0 (0.0)""","""0 (0.0)""","""11 (19.64)""","""0 (0.0)""","""42 (75.0)""","""0 (0.0)""","""45 (80.36)""","""11 (19.64)""",43,0,"""0.66 ± 0.54""","""23.83 ± 19.5""","""2.48 ± 1.36""",56
"""MGH""","""dead""","""58.87 ± 17.5""","""193 (65.2)""","""1 (0.34)""","""5 (1.69)""","""23 (7.77)""","""0 (0.0)""","""0 (0.0)""","""206 (69.59)""","""0 (0.0)""","""62 (20.95)""","""28 (9.46)""","""78 (26.35)""","""190 (64.19)""",275,2,"""0.71 ± 0.47""","""7.37 ± 9.7""","""2.67 ± 2.09""",296
"""BWH""","""not dead""","""57.65 ± 17.19""","""86 (63.24)""","""1 (0.74)""","""4 (2.94)""","""24 (17.65)""","""0 (0.0)""","""0 (0.0)""","""96 (70.59)""","""0 (0.0)""","""11 (8.09)""","""9 (6.62)""","""5 (3.68)""","""122 (89.71)""",111,0,"""0.88 ± 0.49""","""18.59 ± 13.95""","""2.2 ± 1.36""",136
"""BIDMC""","""dead""","""61.2 ± 17.69""","""35 (31.53)""","""0 (0.0)""","""0 (0.0)""","""7 (6.31)""","""0 (0.0)""","""0 (0.0)""","""23 (20.72)""","""0 (0.0)""","""81 (72.97)""","""2 (1.8)""","""84 (75.68)""","""25 (22.52)""",87,0,"""0.55 ± 0.45""","""6.77 ± 5.94""","""3.17 ± 2.84""",111
"""TOTAL""","""not dead""","""58.25 ± 16.92""","""223 (62.64)""","""1 (0.28)""","""12 (3.37)""","""41 (11.52)""","""0 (0.0)""","""0 (0.0)""","""219 (61.52)""","""0 (0.0)""","""83 (23.31)""","""26 (7.3)""","""77 (21.63)""","""253 (71.07)""",296,6,"""0.79 ± 0.52""","""24.37 ± 68.45""","""2.37 ± 1.91""",356
"""TOTAL""","""dead""","""60.2 ± 17.28""","""368 (57.14)""","""1 (0.16)""","""12 (1.86)""","""74 (11.49)""","""0 (0.0)""","""0 (0.0)""","""377 (58.54)""","""0 (0.0)""","""181 (28.11)""","""56 (8.7)""","""198 (30.75)""","""390 (60.56)""",553,2,"""0.7 ± 0.47""","""7.22 ± 8.89""","""2.61 ± 2.08""",644


## error records

In [106]:
err_records = set()
err_late_arrest = df.filter(
    pl.col('out of hospital arrest includes arrest in a rehab facility or SNF') == 'Yes'
).with_columns(
    pl.col('BDSP Patient ID').cast(pl.Int32),
    (pl.col('EEG start') - pl.col('Date of arrest')).alias('time_to_eeg')
).filter(
    pl.col('time_to_eeg') < 0
)
print('ERROR: late arrest (after EEG start)')
display(err_late_arrest)
err_records.update(err_late_arrest['Record ID'].unique())
err_delayed_eeg = df.filter(
    pl.col('out of hospital arrest includes arrest in a rehab facility or SNF') == 'Yes'
).with_columns(
    pl.col('BDSP Patient ID').cast(pl.Int32),
    (pl.col('EEG start') - pl.col('Date of arrest')).alias('time_to_eeg')
).filter(
    pl.col('time_to_eeg') > pl.duration(days=3)
)
print('ERROR: delayed EEG')
display(err_delayed_eeg)
err_records.update(err_delayed_eeg['Record ID'].unique())

ERROR: late arrest (after EEG start)


Record ID,Site,BDSP Patient ID,Gender,Age,MRN (IDENTIFIER),Lastname (IDENTIFIER),Firstname (IDENTIFIER),DOB (IDENTIFIER),Race (choice=American Indian/Alaska),Race (choice=Asian),Race (choice=Black/African American),Race (choice=Hispanic),Race (choice=Hawaiian),Race (choice=White),Race (choice=Mixed),Race (choice=Unknown),Ethnicity,Complete?,Date of arrest,Cardiac Arrest (no circulation),out of hospital arrest includes arrest in a rehab facility or SNF,Cardiac arrest witnessed?,Assisted Ventilation,Temperature,Complete?_duplicated_0,EEG performed,EEG start,EEG end,Total hours on EEG,best background (choice=delta),best background (choice=theta),best background (choice=alpha),All EEG patterns in the 24 hours EEG (choice=Discontinuous EEG),All EEG patterns in the 24 hours EEG (choice=EEG reactivity),All EEG patterns in the 24 hours EEG (choice=Background anytime attenuated 10-20 uV),All EEG patterns in the 24 hours EEG (choice=Background anytime suppressed < 10 uV),…,All EEG patterns in the rest of EEG (choice=Burst suppression with highly epileptiform bursts),All EEG patterns in the rest of EEG (choice=Seizure),All EEG patterns in the rest of EEG (choice=Status epilepticus),All EEG patterns in the rest of EEG (choice=Myoclonic Status epilepticus (time-locked myoclonus and epileptiform discharges)),All EEG patterns in the rest of EEG (choice=GPD),All EEG patterns in the rest of EEG (choice=LPD),All EEG patterns in the rest of EEG (choice=LRDA),All EEG patterns in the rest of EEG (choice=GRDA),All EEG patterns in the rest of EEG (choice=SED),All EEG patterns in the rest of EEG (choice=SIRPDs),All EEG patterns in the rest of EEG (choice=BIRDs),All EEG patterns in the rest of EEG (choice=Diffuse voltage suppression),All EEG patterns in the rest of EEG (choice=BIPD),All EEG patterns in the rest of EEG (choice=Absent EEG reactivity),Antiepileptic drugs[73 hrs to rest of EEG] (choice=Levetiracetam),Antiepileptic drugs[73 hrs to rest of EEG] (choice=Phenytoin-fosphenytoin),Antiepileptic drugs[73 hrs to rest of EEG] (choice=Valproic acid),Antiepileptic drugs[73 hrs to rest of EEG] (choice=Lacosamide),Antiepileptic drugs[73 hrs to rest of EEG] (choice=Clonazepam),Antiepileptic drugs[73 hrs to rest of EEG] (choice=Lorazepam),Antiepileptic drugs[73 hrs to rest of EEG] (choice=Lamotrigine),Antiepileptic drugs[73 hrs to rest of EEG] (choice=Clobazam),Antiepileptic drugs[73 hrs to rest of EEG] (choice=Phenobarbital),Antiepileptic drugs[73 hrs to rest of EEG] (choice=Other),Other anti-epileptic drugs_duplicated_2,Anesthetic[73hrs-rest of EEG] (choice=Propofol),Anesthetic[73hrs-rest of EEG] (choice=Midazolam),Anesthetic[73hrs-rest of EEG] (choice=Fentanyl),Anesthetic[73hrs-rest of EEG] (choice=Dexmedetomidine),Anesthetic[73hrs-rest of EEG] (choice=Other),Other anesthetic drugs_duplicated_2,Complete?_duplicated_1,Discharge Date,CPC at Discharge,Time of Death,Complete?_duplicated_2,time_to_eeg
i64,str,i32,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,datetime[μs],str,str,str,str,str,str,str,datetime[μs],datetime[μs],f64,str,str,str,str,str,str,str,…,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,datetime[μs],str,str,str,duration[μs]


ERROR: delayed EEG


Record ID,Site,BDSP Patient ID,Gender,Age,MRN (IDENTIFIER),Lastname (IDENTIFIER),Firstname (IDENTIFIER),DOB (IDENTIFIER),Race (choice=American Indian/Alaska),Race (choice=Asian),Race (choice=Black/African American),Race (choice=Hispanic),Race (choice=Hawaiian),Race (choice=White),Race (choice=Mixed),Race (choice=Unknown),Ethnicity,Complete?,Date of arrest,Cardiac Arrest (no circulation),out of hospital arrest includes arrest in a rehab facility or SNF,Cardiac arrest witnessed?,Assisted Ventilation,Temperature,Complete?_duplicated_0,EEG performed,EEG start,EEG end,Total hours on EEG,best background (choice=delta),best background (choice=theta),best background (choice=alpha),All EEG patterns in the 24 hours EEG (choice=Discontinuous EEG),All EEG patterns in the 24 hours EEG (choice=EEG reactivity),All EEG patterns in the 24 hours EEG (choice=Background anytime attenuated 10-20 uV),All EEG patterns in the 24 hours EEG (choice=Background anytime suppressed < 10 uV),…,All EEG patterns in the rest of EEG (choice=Burst suppression with highly epileptiform bursts),All EEG patterns in the rest of EEG (choice=Seizure),All EEG patterns in the rest of EEG (choice=Status epilepticus),All EEG patterns in the rest of EEG (choice=Myoclonic Status epilepticus (time-locked myoclonus and epileptiform discharges)),All EEG patterns in the rest of EEG (choice=GPD),All EEG patterns in the rest of EEG (choice=LPD),All EEG patterns in the rest of EEG (choice=LRDA),All EEG patterns in the rest of EEG (choice=GRDA),All EEG patterns in the rest of EEG (choice=SED),All EEG patterns in the rest of EEG (choice=SIRPDs),All EEG patterns in the rest of EEG (choice=BIRDs),All EEG patterns in the rest of EEG (choice=Diffuse voltage suppression),All EEG patterns in the rest of EEG (choice=BIPD),All EEG patterns in the rest of EEG (choice=Absent EEG reactivity),Antiepileptic drugs[73 hrs to rest of EEG] (choice=Levetiracetam),Antiepileptic drugs[73 hrs to rest of EEG] (choice=Phenytoin-fosphenytoin),Antiepileptic drugs[73 hrs to rest of EEG] (choice=Valproic acid),Antiepileptic drugs[73 hrs to rest of EEG] (choice=Lacosamide),Antiepileptic drugs[73 hrs to rest of EEG] (choice=Clonazepam),Antiepileptic drugs[73 hrs to rest of EEG] (choice=Lorazepam),Antiepileptic drugs[73 hrs to rest of EEG] (choice=Lamotrigine),Antiepileptic drugs[73 hrs to rest of EEG] (choice=Clobazam),Antiepileptic drugs[73 hrs to rest of EEG] (choice=Phenobarbital),Antiepileptic drugs[73 hrs to rest of EEG] (choice=Other),Other anti-epileptic drugs_duplicated_2,Anesthetic[73hrs-rest of EEG] (choice=Propofol),Anesthetic[73hrs-rest of EEG] (choice=Midazolam),Anesthetic[73hrs-rest of EEG] (choice=Fentanyl),Anesthetic[73hrs-rest of EEG] (choice=Dexmedetomidine),Anesthetic[73hrs-rest of EEG] (choice=Other),Other anesthetic drugs_duplicated_2,Complete?_duplicated_1,Discharge Date,CPC at Discharge,Time of Death,Complete?_duplicated_2,time_to_eeg
i64,str,i32,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,datetime[μs],str,str,str,str,str,str,str,datetime[μs],datetime[μs],f64,str,str,str,str,str,str,str,…,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,datetime[μs],str,str,str,duration[μs]
